In [10]:
import json
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex

In [22]:
KEYS_TO_EXTRACT = ('participantID', 'timestamp', 'scene', 
    'game.setup', 'game.gameplay', 'game.scoring', 'game.difficulty', 'game.firstTimeScore', 
    'gameScore.score', 'gameScore.thoughts', 
    'debrief.strategy', 'debrief.difficulties', 'debrief.questions', 'debrief.external_aids')

GAME_KEYS = ('setup', 'gameplay', 'scoring', 'difficulty', 'firstTimeScore')

SHORT_SCENE_NAMES = {
    'FloorPlan326_physics_semi_sparse_few_new_objects': 'few_objects',
    'FloorPlan326_physics_semi_sparse_new_objects': 'medium_objects',
    'FloorPlan326_physics_semi_sparse_many_new_objects': 'many_objects',
}

GAME_TEMPLATE = """(define (game game-{index}) (:domain {room}-objects-room-v1)  ; {index}
(:setup (and 

))
(:constraints (and 

))
(:scoring maximize

))"""

def recursive_extract_value(d, key):
    if '.' in key:
        split_key = key.split('.')

    else:
        split_key = [key]

    value = d
    for key_part in split_key:
        if key_part == 'game':
            key_part = 'editedGame' if 'editedGame' in value else 'initialGame'
                
        value = value[key_part] if key_part in value else None
        if value == None:
            return value

    return value


def is_edited_and_fields(doc_dict):
    if 'initialGame' not in doc_dict:
        return None, []

    if 'editedGame' not in doc_dict:
        return False, []

    edited_fields = [key for key in GAME_KEYS if doc_dict['initialGame'][key] != doc_dict['editedGame'][key]]
    return any(edited_fields), edited_fields


def participant_dict_to_row(doc, keys=KEYS_TO_EXTRACT):
    d = doc.to_dict()
    row = {'id': doc.id}
    row.update({key.replace('.', '_'): recursive_extract_value(d, key) for key in keys})
    game_edited, edited_fields = is_edited_and_fields(d)
    row['game_edited'] = game_edited
    row['edited_game_fields'] = ','.join(edited_fields)
    return row


def print_participant(df, index, game_fields=('game_setup', 'game_gameplay', 'game_scoring', 'game_difficulty', 'game_firstTimeScore')):
    p = df.loc[index]
    display(Markdown(f'## #{index} ({p.id}) ({p.scene})'))
    display(Markdown(f'Collected at {p.timestamp}'))

    for game_field in game_fields:
        display(Markdown(f'### {game_field.replace("game_", "")}:'))
        display(Markdown(str(p[game_field])))

    room = p.scene.split('_')[0] if p.scene is not None else ''
    print(GAME_TEMPLATE.format(room=room, index=index))
    print()

    schema_template = dict(metadata=dict(id=p.id, index=index, room=p.scene, notes='')) 
    print(json.dumps(schema_template, indent=4))
    

STATISTICS_CSV_COLUMNS = (
    'id', 'timestamp', 'scene', 
    'game_setup', 'game_gameplay', 'game_scoring', 'game_difficulty',
    'game_firstTimeScore', 'gameScore_score', 'gameScore_thoughts',
    'game_edited', 'edited_game_fields'
)

STATISTICS_CSV_OUT_PATH = '../data/interactive_beta_firestore_statistics.csv'


INDICES_TO_SKIP = (18, 26, 86, 91, 102, 107, )

def df_to_statistics_csv(df, out_path=STATISTICS_CSV_OUT_PATH):
    out_df = df[~df.index.isin(INDICES_TO_SKIP)]
    out_df = out_df.reindex(columns=STATISTICS_CSV_COLUMNS)
    print(out_df.shape, out_df.columns)
    out_df.to_csv(out_path)


In [17]:
LOAD_DATA_FROM_FIRESTORE = False
WRITE_DATA_TO_CSV = False
PARTICIPANTS_CSV_PATH = '../data/interactive_beta.csv'
COLLECTION_NAME = 'participants-v2'


if LOAD_DATA_FROM_FIRESTORE:
    import firebase_admin
    from firebase_admin import credentials
    from firebase_admin import firestore

    # Use a service account
    cred = credentials.Certificate('./game-generation-6db9c-aef76f1917f8.json')
    firebase_admin.initialize_app(cred)

    db = firestore.client()

    participants_with_replays = db.collection(COLLECTION_NAME).order_by('timestamp', direction=firestore.Query.ASCENDING).order_by('replays', direction=firestore.Query.DESCENDING).stream()
    participant_rows = [participant_dict_to_row(doc) for doc in participants_with_replays]
    participant_df = pd.DataFrame(participant_rows)
    participant_df.scene = [SHORT_SCENE_NAMES[s] if s in SHORT_SCENE_NAMES else None for s in participant_df.scene]
    participant_df = participant_df[participant_df.scene.notna() & participant_df.game_setup.notna() & participant_df.game_gameplay.notna() & participant_df.game_scoring.notna()]
    participant_df = participant_df[participant_df.participantID.str.len() > 10]
    participant_df = participant_df[participant_df.participantID != '5f63a8f17e0e2f0c5aebfc0b']
    participant_df = participant_df.reset_index()

    if WRITE_DATA_TO_CSV:
        participant_df.to_csv(PARTICIPANTS_CSV_PATH)
        df_to_statistics_csv(participant_df)

else:
    participant_df = pd.read_csv(PARTICIPANTS_CSV_PATH)



In [18]:
print(participant_df.shape)
participant_df.head()

(119, 19)


,Unnamed: 0.1,Unnamed: 0,index,id,timestamp,scene,game_setup,game_gameplay,game_scoring,game_difficulty,game_firstTimeScore,gameScore_score,gameScore_thoughts,debrief_strategy,debrief_difficulties,debrief_questions,debrief_external_aids,game_edited,edited_game_fields
0,0,0,17,RfP7c4trKFGmDp6xpiG8,2021-10-22 20:57:12.168000+00:00,medium_objects,Place small ramp in front of bin where there i...,"Take ball (any), and try to get it into the bi...",1 point per successful hit.,1,6,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,1,1,18,eYM9NgeUlg9mGpwlGemD,2021-10-22 20:58:20.433000+00:00,many_objects,NaN,The game I can play in this room it is call me...,the scoring system can be the more hits you ha...,1,"from 1 to 10. I would say an 8 haha, I'm not g...",I scored 7,I think it might be better if the objects disa...,Starting by memorizing the shapes of objects a...,"yes, it could be that the game programming is ...",I would like to play the game when it is fully...,no,False,NaN
2,2,2,19,SFpazSkgQ7MFSwDEa3c9,2021-10-22 21:02:15.391000+00:00,many_objects,Open the top drawer beside your bed.,First you pick up a dodgeball or a golf ball a...,To score in this game you just have to throw a...,1,10,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,3,3,20,lchV8TQjaHcYtHAqR31Q,2021-10-23 20:36:08.420000+00:00,many_objects,NaN,Create a tower with the largest number of figu...,Each level of the tower will count as 1 point,3,I would score 6 points,5,It is very hard,I thought as if I were a child playing with to...,No.,It would be nice to let the user change the ke...,"No, just my imagination",False,NaN
4,4,4,21,tM0MidzjBJBLkOEPYr2F,2021-10-23 22:33:04.491000+00:00,medium_objects,The ball must roll down the ramp and hit as ma...,Minni ball bash,The more blocks it hits the more points you get,2,4,2,I dont like this game that mutch,All making use of grafity and placement of the...,No. The game is unprodictable and that was the...,No,No,False,NaN


In [24]:
# df_to_statistics_csv(participant_df)

# participant_df.loc[participant_df.participantID == '5bc79f652885710001a0e82a', :]

def word_in_game(df, word):
    return np.logical_or([word in g for g in df.game_setup], np.logical_or([word in g for g in df.game_gameplay], [word in g for g in df.game_scoring]))


def game_to_format(game):
    return f'SETUP: {game["game_setup"]} GAMEPLAY: {game["game_gameplay"]} SCORING: {game["game_scoring"]}'



# game_to_format(participant_df.loc[word_in_game(participant_df, 'pink zone'), :].iloc[0])

In [25]:
print_participant(participant_df, 111) 

## #111 (ZBcXIZbvTS3U4IBGk1zk) (medium_objects)

Collected at 2022-01-24 00:49:09.530000+00:00

### setup:

Balls
Bin
Small Ramp
One player

### gameplay:

...

### difficulty:

4

### firstTimeScore:

0

(define (game game-111) (:domain medium-objects-room-v1)  ; 111
(:setup (and 

))
(:constraints (and 

))
(:scoring maximize

))

{
    "metadata": {
        "id": "ZBcXIZbvTS3U4IBGk1zk",
        "index": 111,
        "room": "medium_objects",
        "notes": ""
    }
}


In [121]:
participant_df[~participant_df.index.isin(INDICES_TO_SKIP)].shape

(113, 19)

In [107]:
duplicate_ids = set(participant_df.participantID[participant_df.participantID.duplicated()])


participant_df.loc[[pid in duplicate_ids for pid in participant_df.participantID], :]

,Unnamed: 0,index,id,participantID,timestamp,scene,game_setup,game_gameplay,game_scoring,game_difficulty,game_firstTimeScore,gameScore_score,gameScore_thoughts,debrief_strategy,debrief_difficulties,debrief_questions,debrief_external_aids,game_edited,edited_game_fields
17,17,54,GLPtcvJUaHkUYK7iEPRq,613e4bf960ca68f8de00e5e7,2021-11-08 21:14:13.145000+00:00,medium_objects,NaN,The pieces on the shelf between the two window...,For each castle built in the correct order of ...,0,40,0,In real life this game would be very easy (tha...,NaN,NaN,NaN,NaN,False,NaN
18,18,55,PJ2WCWCLedT8sDLrvn49,613e4bf960ca68f8de00e5e7,2021-11-08 21:55:19.879000+00:00,medium_objects,NaN,The pieces on the shelf between the two Window...,For each castle built in the correct order wil...,1,40,0,In real life this game would be very easy (tha...,I just looked at what was in the room and deci...,"Yes, the page broke and I had to redo it.",No,Print on a piece of paper the shape of the cas...,True,"gameplay,scoring"
25,25,72,zhq2iVuBVQxs15gj7Blw,605e14202c3a566911baccda,2021-12-22 16:58:43.828000+00:00,many_objects,To prepare the room for the game you need to h...,To play my game you have to find all the pyram...,You will get a point for every pyramid you wil...,0,I would score the maximum points - 6 points (f...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
26,26,73,cpECU5M34mXo0nVEGZka,605e14202c3a566911baccda,2021-12-22 17:14:59.558000+00:00,few_objects,You have to hide all the cubes,To play this game you have to find all the cub...,You will get a point for every cube you will m...,0,I would score maximum points > 6 points (for 6...,I scored 6 points for those 6 cubes that I hav...,Now that I payed my game I found out it is eas...,I found objects that would be similar to each ...,"Yes, I had to swiitch from my old Mac laptop t...","It was a very interesting survey, thank you! A...","I didn't use anything, because my game was not...",False,NaN
84,84,160,jCc0kkmGUg3xUmUSXg5w,60306cf6330619ee41fa3cd2,2022-01-22 18:46:24.805000+00:00,few_objects,"To set up the game, take the numerous small ob...",The game is a 'find the object' game where you...,"There are seven objects (the CD, watch, phone,...",1,550,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
86,86,162,QclKeEZEVr7j0klPuanE,60306cf6330619ee41fa3cd2,2022-01-22 19:15:36.983000+00:00,few_objects,"Begin by placing the small objects (CD, watch,...",This is a 'find the objects' game in a small r...,Each object is worth 100 points upon finding i...,1,550,474,The objects are clipping and disappearing pret...,I looked about and thought maybe I could do so...,"Yes, the program closed itself out on my brows...","I hope that my game, though easy and a bit fau...",I wrote down the objects I had in mind on a no...,False,NaN
89,89,167,ktwB7wT09sh4ivNme3Dw,6103ec2bf88328284fd894bc,2022-01-22 22:03:24.545000+00:00,medium_objects,Player has to take the monitor off the table u...,Player has to pick up one of the three balls t...,"Scoring Dodgeball grants +1 point, scoring bas...",4,4-6 points,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
91,91,169,IhOkh1l3TBY9JJVubzHx,6103ec2bf88328284fd894bc,2022-01-22 22:23:14.714000+00:00,many_objects,Monitor has to be taken off the table using ot...,Player is supposed to throw balls inside the b...,Each dodgeball counts as one point. Getting th...,3,3,I scored nothing,Now that I've played my game I see that score ...,I tried to find objects that could help with s...,Controls are poorly made and I had difficultie...,Improve control system. It is frustrating as o...,I did not use anything additional.,True,"scoring,firstTimeScore"
101,101,184,T2dYV1nPh5cmt3NBELdS,61093eae2bc2e47e6f26c7d7,2022-01-23 15:39:23.007000+00:00,few_objects,"the bin needs to be ontop op the bed,plcae the...",the balls will be used to try and throw them i...,If you throw a ball in from behind the blue bl...,3,5 points,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
102,102,185,Q6a8AbiIdcLA9tJzAu14,61093eae2bc2e47e6f26c7d7,2022-01-23 16:14:08.311000+00:00,many_objects,"pl

# Statistics on plaintext versions

In [6]:
import textstat
import tabulate

DEFAULT_STATS_FIELDS = ('game_setup', 'game_gameplay', 'game_scoring')

def run_textstat_func(textstat_func, df, fields=DEFAULT_STATS_FIELDS):
    if isinstance(fields, slice):
        subset = df.iloc[:, fields].copy()
    else:
        subset = df.loc[:, fields].copy()
        
    subset[subset.isna()] = ''
    s = subset.agg('\n'.join, axis=1)
    scores = s.apply(textstat_func)
    return scores.mean(), scores.std() / (len(scores) ** 0.5)

In [7]:
run_textstat_func(textstat.flesch_reading_ease, participant_df)

(75.63380952380952, 4.418337517351245)

In [8]:
few_objects_df = pd.read_csv('../data/few_objects.csv')
medium_objects_df = pd.read_csv('../data/medium_objects.csv')
many_objects_df = pd.read_csv('../data/many_objects.csv')

In [9]:
textstat.flesch_reading_ease.__name__

'flesch_reading_ease'

In [11]:
TEXTSTAT_FUNCS = (textstat.flesch_reading_ease, textstat.flesch_kincaid_grade, textstat.gunning_fog)
NAMES = ['survey - few objects', 'survey - medium objects', 'survey - many objects', 'interactive beta']

rows = []

for i, df in enumerate((few_objects_df, medium_objects_df, many_objects_df, participant_df)):
    scores = [run_textstat_func(func, df, fields=slice(1, 4) if i < 3 else DEFAULT_STATS_FIELDS) for func in TEXTSTAT_FUNCS]
    rows.append([NAMES[i]] + [f'$ {s[0]:.2f} \\pm {s[1]:.2f} $' for s in scores])

headers = ['name'] + [func.__name__ for func in TEXTSTAT_FUNCS]

display(Markdown(tabulate.tabulate(rows, headers=headers, tablefmt='github')))

# few_objects_df.iloc[:, slice(1, 4)].agg(lambda x: [type(z) for z in x], axis=1)

| name                    | flesch_reading_ease   | flesch_kincaid_grade   | gunning_fog        |
|-------------------------|-----------------------|------------------------|--------------------|
| survey - few objects    | $ 61.48 \pm 7.66 $    | $ 14.73 \pm 2.96 $     | $ 17.11 \pm 3.06 $ |
| survey - medium objects | $ 61.76 \pm 5.55 $    | $ 13.73 \pm 2.08 $     | $ 16.03 \pm 2.12 $ |
| survey - many objects   | $ 56.42 \pm 5.95 $    | $ 15.56 \pm 2.23 $     | $ 17.65 \pm 2.30 $ |
| interactive beta        | $ 75.63 \pm 4.42 $    | $ 9.39 \pm 1.59 $      | $ 11.83 \pm 1.57 $ |